In [1]:
%matplotlib inline

In [18]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import time
import os

In [7]:
data_path = "../../../FB15k/"

In [4]:
def calc_sum(list1, list2):
    s = 0;
    for i in xrange(len(list1)):
        s += abs(list1[i] - list2[i])
    return abs(s)

def parseline(line):
    vec = []
    for i in line.split("\t"):
        if i != "\n":
            vec += [float(i)]
    return vec
def read_vec(path, file_name):
    vec = []
    f = open(path+file_name, 'r')
    data = f.readlines()
    f.close()
    for i in data:
        vec += [parseline(i)]
    return vec
def read_data(path, file_name):
    list1 = []
    dict1 = {}
    count = 0
    f = open(path+file_name, 'r')
    data = f.readlines()
    f.close()
    for i in data:
        str1, num = i.split("\t")
        list1 += [str1]
        dict1[str1] = count
        count += 1
    return list1, dict1

## read result vec

In [6]:
# entity_vec = read_vec('../', "entity2vec.bern")
# relation_vec = read_vec('../', "relation2vec.bern")
with open('../entity2vec.bern', 'r') as f:
    entity_vec = pickle.load(f)
with open('../relation2vec.bern', 'r') as f:
    relation_vec = pickle.load(f)

## read training data

In [8]:
entitylist, entity2id = read_data(data_path, 'entity2id.txt')
relationlist, relation2id = read_data(data_path, 'relation2id.txt')

## read freebase to wikidata

In [9]:
f = open(data_path+'f2w.nt', "r")
data = f.readlines()
f.close()
fb2wiki = {}
wiki2fb = {}
for i in data:
    fb, wiki = i.split(' ')
    fb2wiki[fb] = wiki.rstrip()
    wiki2fb[wiki.rstrip()] = fb

## input search string

In [10]:
inp_str = raw_input("input a string:")
res = requests.get("https://www.wikidata.org/w/index.php?search=&search="+inp_str+"&title=Special%3ASearch&go=Go")
soup = BeautifulSoup(res.text,"html.parser")
for tag in soup.find_all("span", class_="wb-itemlink-id"):
	inp_str = tag.string
	print "search first id: ", inp_str
	break
for tag in soup.find_all("span",  class_="wb-itemlink-label"):
	print "search first result: ",tag.string
	break

input a string:beijing
search first id:  (Q956)
search first result:  Beijing


In [11]:
target = entity2id[wiki2fb[inp_str[1:len(inp_str)-1]]]
result = {}
K = 10

## calcuate the sum and sort

In [12]:
def find_top_k(ii, target):
    candidate_list = {}
    print relationlist[ii]
    for i in xrange(len(entitylist)):
    #	for ii in xrange(len(relationvec)):
        try:
            candidate_list[fb2wiki[entitylist[i]]] = calc_sum(np.add(entity_vec[target], relation_vec[ii]), entity_vec[i])
        #	candidate_list[fb2wiki[entitylist[i]]] = calc_sum(entityvec[k], entityvec[i])
        except KeyError:
            pass
    # find the top k predictions
    count = 0
    simlist = []
    f = open('cost.txt', 'a+')
    f.write(relationlist[ii]+':\n')
    for i in sorted(candidate_list.items(), key= lambda item:item[1]):
        simlist += [i[0]]
        f.write(i[0] + '\t' + str(i[1]))
        count += 1
        if count > K:
            break
    return simlist

## search on wikidata for the meaning of the index

In [13]:
def search_wikidata(simlist):
    dict1 = {}
    for i in simlist:
        res = requests.get("https://www.wikidata.org/wiki/"+i)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text,"html.parser")
        for tag in soup.find_all("span", class_="wikibase-title-label"):
             label = tag.string
        for tag in soup.find_all("span", class_="wikibase-descriptionview-text"):	
            desc = tag.string
        dict1[label] = desc
    return dict1

In [ ]:
t1 = time.time()
start = 0 
if os.path.exists("timeout.txt"):
    with open('timeout.txt', 'r') as f:
        start = int(f.readlines())
for i in range(start, len(relation2id)):
    simlist = find_top_k(i, target)
    try:
        result[relationlist[i]] = search_wikidata(simlist)
    except requests.exceptions.ConnectTimeout:
        with open('timeout.txt','w') as f:
            f.write(start)
        with open('result_dict.txt', 'w') as f:
            pickle.dump(result, f)
        exit(0)
    except BaseExceptioin, e:
        print e
        time.sleep(1)

/people/appointed_role/appointment./people/appointment/appointed_by
/location/statistical_region/rent50_2./measurement_unit/dated_money_value/currency
/tv/tv_series_episode/guest_stars./tv/tv_guest_role/actor
/music/performance_role/track_performances./music/track_contribution/contributor
/medicine/disease/prevention_factors
/organization/organization_member/member_of./organization/organization_membership/organization
/american_football/football_player/receiving./american_football/player_receiving_statistics/season


In [72]:
res = pd.DataFrame(result)

In [73]:
res.head()

""
